In [2]:
from statistics import stdev
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from os import walk, makedirs
import pandas as pd
import numpy as np

predictions_dir = '../predictions/'
# k_folds_dirs = next(walk(predictions_dir))[1]
baseline_dirnames = ['Bert_embeddings', 'TFidf']
result_cols=["model", "traindata", "accuracy", "std_accuracy", "precision", "std_precision", "recall", "std_recall", "f1", "std_f1"]
dfs = {'Bert_embeddings':pd.DataFrame(columns=result_cols),'TFidf':pd.DataFrame(columns=result_cols)}

for baseline in baseline_dirnames:
  df = dfs.get(baseline)
  model_dirs = next(walk(f"{predictions_dir}/k_is_0/{baseline}"))[1]
  for model in model_dirs:
    result_files = next(walk(f"{predictions_dir}/k_is_0/{baseline}/{model}"))[2]
    for file in result_files: # 'text', 'class', 'predicted'
      prec = list()
      recall = list()
      f1 = list()
      accuracy = list()
      
      for k in range(0,3):
        results = pd.read_csv(f"{predictions_dir}/k_is_{k}/{baseline}/{model}/{file}").sample(frac=1)
        prec_curr, recall_curr, f1_curr, _ = precision_recall_fscore_support(results["class"], results["predicted"], average='macro')
        accuracy_curr = accuracy_score(results["class"], results["predicted"])
        prec.append(prec_curr)
        recall.append(recall_curr)
        f1.append(f1_curr)
        accuracy.append(accuracy_curr)
      
      # add results to dict
      df.loc[len(df.index)] = [model, file,
                              np.mean(accuracy), stdev(accuracy),
                              np.mean(prec), stdev(prec),
                              np.mean(recall), stdev(recall),
                              np.mean(f1), stdev(f1)]

ModuleNotFoundError: No module named 'sklearn'

In [3]:
results_dir="../results"
for baseline in baseline_dirnames:
  dfs[baseline].to_csv(f"{results_dir}/{baseline}", index=False)

##### make the latex results chart

In [ ]:
latexCols=["model","2quads","4quads","8quads","16quads","22quads","28quads"]
modelNames=list()
trainNames=["train2","train4","train8","train16","train22","train28"]

for baseline in baseline_dirnames:
  modelNames=df.model.unique() #get modelnames
  latex_df=pd.DataFrame(columns=latexCols)
  latex_df.model = modelNames
  # for index, row in latex_df.iterrows():
  #   latex_df.at[2,'age']
  
